In [1]:
# Check the current version of Keras
!pip show tensorflow
!pip show keras

# Uninstall the current version of Keras
!pip uninstall tensorflow -y
!pip uninstall keras -y

# Install the specific version of Keras
!pip install tensorflow==2.15.1
# !pip install keras==2.7.0

Name: tensorflow
Version: 2.17.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, requests, setuptools, six, tensorboard, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tf_keras
Name: keras
Version: 3.4.1
Summary: Multi-backend Keras.
Home-page: https://github.com/keras-team/keras
Author: Keras team
Author-email: keras-users@googlegroups.com
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: absl-py, h5py, ml-dtypes, namex, numpy, optree, packaging, rich
Required-by: tensorflow
Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfull

In [2]:
import keras
print(keras.__version__)

2.15.0


In [3]:
import tensorflow
print(tensorflow.__version__)

2.15.1


In [4]:
# !pip install keras

In [5]:
# !pip install np_utils

In [6]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.4 MB/s eta 0:00:00


In [7]:
import numpy as np

import pickle

from scipy import stats

import xgboost as xgb
from xgboost import XGBClassifier

from datetime import datetime

from sklearn.metrics import accuracy_score

import keras
from keras import regularizers
from keras import utils
from keras import optimizers
from keras.models import Sequential
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras.callbacks import LearningRateScheduler
from keras.models import model_from_json
from keras.models import load_model
from tensorflow.keras.utils import to_categorical


import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
import tensorflow.keras as keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import load_model, model_from_json

import optuna

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Tidak dipakai

In [ ]:
# import os
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras import layers, models, optimizers, Sequential
# from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D

# # Load and preprocess the Intel Image Dataset
# base_dir = '/content/drive/MyDrive/intel'

# train_dir = os.path.join(base_dir, 'seg_train/seg_train')
# test_dir = os.path.join(base_dir, 'seg_test/seg_test')

# # ImageDataGenerator for data augmentation and normalization
# train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
# test_datagen = ImageDataGenerator(rescale=1./255)

# train_generator = train_datagen.flow_from_directory(
#     train_dir,
#     target_size=(200, 200),
#     batch_size=16,
#     class_mode='categorical',
#     subset='training'
# )

# validation_generator = train_datagen.flow_from_directory(
#     train_dir,
#     target_size=(200, 200),
#     batch_size=16,
#     class_mode='categorical',
#     subset='validation'
# )

# test_generator = test_datagen.flow_from_directory(
#     test_dir,
#     target_size=(200, 200),
#     batch_size=16,
#     class_mode='categorical'
# )

# conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

# # Freeze the convolutional base
# conv_base.trainable = False

# model = Sequential()
# model.add(conv_base)
# model.add(GlobalAveragePooling2D())
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.50))
# model.add(Dense(256, activation='relu'))
# model.add(Dropout(0.50))
# model.add(Dense(6, activation='softmax'))

# # Use Adam optimizer for faster convergence
# model.compile(
#     optimizer=optimizers.Adam(learning_rate=2e-4),
#     loss='categorical_crossentropy',
#     metrics=['acc']
# )

# print('Build and save ResNet-50 model')

# history = model.fit(
#     train_generator,
#     epochs=50,
#     validation_data=validation_generator,
#     steps_per_epoch=train_generator.samples // train_generator.batch_size,
#     validation_steps=validation_generator.samples // validation_generator.batch_size
# )

# model_json = model.to_json()
# with open('model_resnet.json', 'w') as json_file:
#     json_file.write(model_json)

# model.save_weights('model_resnet.h5')


In [9]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

seg_train = '/content/drive/MyDrive/intel/seg_train/seg_train'
seg_test = '/content/drive/MyDrive/intel/seg_test/seg_test/'
seg_pred = '/content/drive/MyDrive/intel/seg_pred/seg_pred/'

generate = ImageDataGenerator(rescale=1./255,
                              shear_range=0.2,
                              zoom_range=0.2,
                              horizontal_flip=True)
training_set = generate.flow_from_directory(seg_train,
                                            target_size=(100, 100),
                                            batch_size=256,  # Mengurangi batch size
                                            classes=["buildings","forest","glacier","mountain","sea","street"],
                                            class_mode='categorical')
test_set = generate.flow_from_directory(seg_test,
                                        target_size=(100, 100),
                                        batch_size=256,  # Mengurangi batch size
                                        classes=["buildings","forest","glacier","mountain","sea","street"],
                                        class_mode='categorical')

# Build the model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(100, 100, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(256, activation='relu'))  # Mengurangi jumlah neuron
model.add(Dropout(0.5))
model.add(Dense(6, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Callbacks untuk mengurangi waktu pelatihan
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

# Fit the model
model.fit(training_set, epochs=15, verbose=1, validation_data=test_set, callbacks=[early_stopping, reduce_lr])

# Evaluate the model
score = model.evaluate(test_set, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Predict the labels for the test set
y_pred = model.predict(test_set)

# Convert predictions and true labels to class indices
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = test_set.classes

# Generate classification report
report = classification_report(y_true_classes, y_pred_classes, target_names=["buildings","forest","glacier","mountain","sea","street"])
print(report)


Found 14044 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Epoch 1/15
55/55 [==============================] - 4493s 82s/step - loss: 1.2350 - accuracy: 0.4989 - val_loss: 0.9839 - val_accuracy: 0.6133 - lr: 0.0010
Epoch 2/15
55/55 [==============================] - 108s 2s/step - loss: 0.9199 - accuracy: 0.6465 - val_loss: 0.8400 - val_accuracy: 0.6823 - lr: 0.0010
Epoch 3/15
55/55 [==============================] - 104s 2s/step - loss: 0.8157 - accuracy: 0.7001 - val_loss: 0.7970 - val_accuracy: 0.6930 - lr: 0.0010
Epoch 4/15
55/55 [==============================] - 106s 2s/step - loss: 0.7292 - accuracy: 0.7368 - val_loss: 0.6763 - val_accuracy: 0.7457 - lr: 0.0010
Epoch 5/15
55/55 [==============================] - 103s 2s/step - loss: 0.6748 - accuracy: 0.7559 - val_loss: 0.6240 - val_accuracy: 0.7727 - lr: 0.0010
Epoch 6/15
55/55 [==============================] - 106s 2s/step - loss: 0.6011 - accuracy: 0.7859 - val_loss: 0.7051 - val_accuracy: 0.7460 - 

In [ ]:
# # Memuat dataset CIFAR-10
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()

# train_size = int(1 * x_train.shape[0])
# indices = np.random.permutation(x_train.shape[0])
# x_train = x_train[indices][:train_size]
# y_train = y_train[indices][:train_size]

# x_train = x_train / 255.0
# x_test = x_test / 255.0

# y_train = to_categorical(y_train, 10)
# y_test = to_categorical(y_test, 10)

# print('CIFAR-10 dataset loaded')

# conv_base = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(200, 200, 3))
# model = models.Sequential()
# model.add(layers.UpSampling2D((2,2), input_shape=(32, 32, 3)))
# model.add(layers.UpSampling2D((2,2)))
# model.add(layers.Resizing(200, 200))
# model.add(conv_base)
# model.add(layers.Flatten(name='flatten'))  # Menambahkan nama ke layer Flatten
# model.add(layers.BatchNormalization())
# model.add(layers.Dense(128, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.BatchNormalization())
# model.add(layers.Dense(64, activation='relu'))
# model.add(layers.Dropout(0.5))
# model.add(layers.BatchNormalization())
# model.add(layers.Dense(10, activation='softmax'))

# model.compile(
#     optimizer=optimizers.RMSprop(learning_rate=2e-5),
#     loss='binary_crossentropy',
#     metrics=['accuracy']
# )

# print('Build and save ResNet-50 model')

# history = model.fit(x_train, y_train,
#     epochs=5,
#     batch_size=20,
#     validation_data=(x_test, y_test))

# # Menyimpan arsitektur model ke file JSON
# model_json = model.to_json()
# with open('model_resnet.json', 'w') as json_file:
#     json_file.write(model_json)

# # Menyimpan bobot model ke file HDF5 dengan ekstensi yang benar
# model.save_weights('model_resnet.weights.h5')

In [10]:
model_json = model.to_json()
with open('model_resnet.json', 'w') as json_file:
	json_file.write(model_json)
model.save_weights('model_resnet.h5')

In [ ]:
# # Evaluate the model on the test data and calculate additional metrics
# test_steps = test_generator.samples // test_generator.batch_size
# test_generator.reset()
# predictions = model.predict(test_generator, steps=test_steps)
# y_pred = np.argmax(predictions, axis=1)
# y_true = test_generator.classes[:len(y_pred)]

# # Calculate metrics
# accuracy = accuracy_score(y_true, y_pred)
# precision = precision_score(y_true, y_pred, average='weighted')
# recall = recall_score(y_true, y_pred, average='weighted')
# f1 = f1_score(y_true, y_pred, average='weighted')

# print(f'Accuracy: {accuracy:.4f}')
# print(f'Precision: {precision:.4f}')
# print(f'Recall: {recall:.4f}')
# print(f'F1 Score: {f1:.4f}')

In [ ]:
# import os
# import numpy as np
# import tensorflow as tf
# from tensorflow.keras.models import model_from_json
# from tensorflow.keras import optimizers
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import xgboost as xgb
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# import pickle

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# def load_cnn_model(X_test, y_test):
#     json_file = open('model_resnet.json', 'r')
#     loaded_model_json = json_file.read()
#     json_file.close()
#     model = model_from_json(loaded_model_json)
#     model.load_weights('model_resnet.h5')  # load weights into new model

#     model.compile(
#         optimizer=optimizers.RMSprop(learning_rate=2e-5),
#         loss='categorical_crossentropy',
#         metrics=['accuracy']
#     )

#     return model

# def get_feature_layer(model, data):
#     total_layers = len(model.layers)
#     fl_index = total_layers - 7  # Get the correct index for the feature layer

#     feature_layer_model = tf.keras.Model(
#         inputs=model.input,
#         outputs=model.get_layer(index=fl_index).output
#     )
#     feature_layer_output = feature_layer_model.predict(data)

#     return feature_layer_output

# def xgb_model(X_train, y_train, X_test, y_test):
#     y_train = y_train.flatten()
#     y_test = y_test.flatten()

#     dtrain = xgb.DMatrix(X_train, label=y_train)
#     dtest = xgb.DMatrix(X_test, label=y_test)

#     params = {
#         'max_depth': 12,
#         'eta': 0.05,
#         'objective': 'multi:softprob',
#         'num_class': 6,
#         'early_stopping_rounds': 5,
#         'eval_metric': 'merror'
#     }

#     watchlist = [(dtrain, 'train'), (dtest, 'eval')]
#     n_rounds = 150

#     model = xgb.train(params, dtrain, n_rounds, evals=watchlist)

#     return model

# def evaluate_model(model, X_test, y_test):
#     dtest = xgb.DMatrix(X_test)
#     y_pred_prob = model.predict(dtest)
#     y_pred = np.argmax(y_pred_prob, axis=1)

#     accuracy = accuracy_score(y_test, y_pred)
#     precision = precision_score(y_test, y_pred, average='macro')
#     recall = recall_score(y_test, y_pred, average='macro')
#     f1 = f1_score(y_test, y_pred, average='macro')

#     print(f'Accuracy: {accuracy}')
#     print(f'Precision: {precision}')
#     print(f'Recall: {recall}')
#     print(f'F1 Score: {f1}')

#     return accuracy, precision, recall, f1

# def load_intel_image_dataset(base_dir, img_size):
#     datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

#     train_generator = datagen.flow_from_directory(
#         base_dir,
#         target_size=(img_size, img_size),
#         batch_size=32,
#         class_mode='categorical',
#         subset='training'
#     )

#     validation_generator = datagen.flow_from_directory(
#         base_dir,
#         target_size=(img_size, img_size),
#         batch_size=32,
#         class_mode='categorical',
#         subset='validation'
#     )

#     X_train, y_train = next(train_generator)
#     for i in range(1, len(train_generator)):
#         x, y = next(train_generator)
#         X_train = np.concatenate((X_train, x))
#         y_train = np.concatenate((y_train, y))

#     X_test, y_test = next(validation_generator)
#     for i in range(1, len(validation_generator)):
#         x, y = next(validation_generator)
#         X_test = np.concatenate((X_test, x))
#         y_test = np.concatenate((y_test, y))

#     return (X_train, y_train), (X_test, y_test)

# def main():
#     # Load Intel Image Dataset
#     base_dir = '/content/drive/MyDrive/intel'
#     img_size = 150
#     (X_train, y_train), (X_test, y_test) = load_intel_image_dataset(base_dir, img_size)

#     # Flatten the labels
#     y_train = np.argmax(y_train, axis=1)
#     y_test = np.argmax(y_test, axis=1)

#     # Load the CNN model and extract features
#     cnn_model = load_cnn_model(X_test, y_test)

#     X_train_cnn = get_feature_layer(cnn_model, X_train)  # Extract features
#     X_test_cnn = get_feature_layer(cnn_model, X_test)  # Extract features

#     # Train XGBoost model with correct objective function and label format
#     model = xgb_model(X_train_cnn, y_train, X_test_cnn, y_test)

#     print("Build and save of XGBoost Model")
#     pickle.dump(model, open("cnn_xgboost_resnet_final.pickle.dat", "wb"))

#     # Evaluate the model
#     evaluate_model(model, X_test_cnn, y_test)

# if __name__ == '__main__':
#     main()


In [11]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import model_from_json
from keras import optimizers
import tensorflow as tf
import xgboost as xgb
import pickle

def load_cnn_model():
    # Load model architecture
    with open('model_resnet.json', 'r') as json_file:
        loaded_model_json = json_file.read()

    # Load model weights
    model = model_from_json(loaded_model_json)
    model.load_weights('model_resnet.h5')

    # Compile the model
    model.compile(
        optimizer=optimizers.RMSprop(learning_rate=2e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def get_feature_layer(model, data):
    # Get the feature extraction layer
    feature_layer_model = tf.keras.Model(
        inputs=model.input,
        outputs=model.layers[-7].output
    )

    # Extract features
    feature_layer_output = feature_layer_model.predict(data)
    return feature_layer_output

def xgb_model(X_train, y_train, X_test, y_test):
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    params = {
        'max_depth': 12,
        'eta': 0.05,
        'objective': 'multi:softprob',
        'num_class': 6,
        'eval_metric': 'merror'
    }

    model = xgb.train(params, dtrain, num_boost_round=150, evals=[(dtrain, 'train'), (dtest, 'eval')])
    return model

def evaluate_model(model, X_test, y_test):
    dtest = xgb.DMatrix(X_test)
    y_pred_prob = model.predict(dtest)
    y_pred = np.argmax(y_pred_prob, axis=1)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')

    print(f'Accuracy: {accuracy}')
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')

    return accuracy, precision, recall, f1

def load_intel_image_dataset(base_dir, img_size):
    datagen = ImageDataGenerator(rescale=1.0/255.0, validation_split=0.2)

    train_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(img_size, img_size),
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        base_dir,
        target_size=(img_size, img_size),
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )

    train_dataset = tf.data.Dataset.from_generator(
        lambda: train_generator,
        output_signature=(
            tf.TensorSpec(shape=(None, img_size, img_size, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, train_generator.num_classes), dtype=tf.float32)
        )
    ).prefetch(buffer_size=tf.data.AUTOTUNE)

    validation_dataset = tf.data.Dataset.from_generator(
        lambda: validation_generator,
        output_signature=(
            tf.TensorSpec(shape=(None, img_size, img_size, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, validation_generator.num_classes), dtype=tf.float32)
        )
    ).prefetch(buffer_size=tf.data.AUTOTUNE)

    X_train, y_train = [], []
    for x, y in train_dataset:
        X_train.append(x)
        y_train.append(y)

    X_train = np.concatenate(X_train)
    y_train = np.concatenate(y_train)

    X_test, y_test = [], []
    for x, y in validation_dataset:
        X_test.append(x)
        y_test.append(y)

    X_test = np.concatenate(X_test)
    y_test = np.concatenate(y_test)

    return (X_train, y_train), (X_test, y_test)

def main():
    # Load Intel Image Dataset
    base_dir = '/content/drive/MyDrive/intel/'
    img_size = 150
    (X_train, y_train), (X_test, y_test) = load_intel_image_dataset(base_dir, img_size)

    # Convert labels from one-hot to single dimension
    y_train = np.argmax(y_train, axis=1)
    y_test = np.argmax(y_test, axis=1)

    # Load the CNN model
    cnn_model = load_cnn_model()

    # Extract features using the CNN model
    X_train_cnn = get_feature_layer(cnn_model, X_train)
    X_test_cnn = get_feature_layer(cnn_model, X_test)

    # Train XGBoost model
    xgb_model_trained = xgb_model(X_train_cnn, y_train, X_test_cnn, y_test)

    # Save the XGBoost model
    with open("cnn_xgboost_resnet_final.pickle.dat", "wb") as f:
        pickle.dump(xgb_model_trained, f)

    # Evaluate the model
    evaluate_model(xgb_model_trained, X_test_cnn, y_test)

if __name__ == '__main__':
    main()


Found 19513 images belonging to 3 classes.
Found 4877 images belonging to 3 classes.


KeyboardInterrupt: 

In [ ]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from keras.models import model_from_json
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope

feature_extractor = Model(inputs=model.input, outputs=model.layers[-3].output)
train_features = feature_extractor.predict(training_set)
test_features = feature_extractor.predict(test_set)

# Get labels
train_labels = training_set.classes
test_labels = test_set.classes

# Split data for hyperparameter tuning
X_train, X_val, y_train, y_val = train_test_split(train_features, train_labels, test_size=0.2, random_state=42)

# Define objective function for TPE
def objective(params):
    model = XGBClassifier(
        **{k: int(v) if k in ['max_depth', 'n_estimators', 'min_child_weight', 'num_parallel_tree'] else v for k, v in params.items()},
        use_label_encoder=False,
        eval_metric='mlogloss'
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    accuracy = np.mean(y_pred == y_val)
    return {'loss': -accuracy, 'status': STATUS_OK}

# Define hyperparameter space
space = {
    'learning_rate': hp.uniform('learning_rate', 0.001, 0.1),
    'max_depth': hp.quniform('max_depth', 1, 15, 1),
    'min_child_weight': hp.uniform('min_child_weight', 0, 5),
    'subsample': hp.uniform('subsample', 0.1, 1),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1),
    'base_score': hp.uniform('base_score', 0.1, 0.9),
    'colsample_bylevel': hp.uniform('colsample_bylevel', 0.1, 1),
    'colsample_bynode': hp.uniform('colsample_bynode', 0.1, 1),
    'max_delta_step': hp.uniform('max_delta_step', 0, 1),
    'num_parallel_tree': hp.quniform('num_parallel_tree', 1, 10, 1),
    'gamma': hp.uniform('gamma', 0, 0.1)
}

# Run TPE optimization
trials = Trials()
best_params = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=10, trials=trials)

# Convert best_params to proper types
best_params = {k: int(v) if k in ['max_depth', 'n_estimators', 'min_child_weight', 'num_parallel_tree'] else v for k, v in best_params.items()}

# Train final model with best parameters
final_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='mlogloss')
final_model.fit(train_features, train_labels)

# Predict and evaluate
y_pred = final_model.predict(test_features)
accuracy = np.mean(y_pred == test_labels)
print('Test accuracy:', accuracy)

# Generate classification report
report = classification_report(test_labels, y_pred, target_names=["buildings","forest","glacier","mountain","sea","street"])
print(report)

# Save the final model
final_model.save_model('xgboost_model.json')

  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:26:33] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



 10%|█         | 1/10 [02:29<22:29, 149.92s/trial, best loss: -0.16411534353862584]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:29:03] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



 20%|██        | 2/10 [08:57<38:36, 289.57s/trial, best loss: -0.18120327518689924]

...

 90%|█████████ | 9/10 [34:15<02:25, 145.45s/trial, best loss: -0.18120327518689924]

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:00:48] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



100%|██████████| 10/10 [34:34<00:00, 207.49s/trial, best loss: -0.18120327518689924]


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [05:01:08] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test accuracy: 0.161
              precision    recall  f1-score   support

   buildings       0.16      0.13      0.14       437
      forest       0.16      0.14      0.15       474
     glacier       0.16      0.16      0.16       553
    mountain       0.17      0.23      0.20       525
         sea       0.16      0.15      0.15       510
      street       0.15      0.15      0.15       501

    accuracy                           0.16      3000
   macro avg       0.16      0.16      0.16      3000
weighted avg       0.16      0.16      0.16      3000



In [ ]:
best_params['max_depth'] = int(best_params['max_depth'])
best_params['n_estimators'] = int(best_params['n_estimators'])
best_params['min_child_weight'] = int(best_params['min_child_weight'])

final_model = XGBClassifier(**best_params, use_label_encoder=False, eval_metric='mlogloss')
final_model.fit(train_features, train_labels)

# Predict and evaluate
y_pred = final_model.predict(test_features)
accuracy = np.mean(y_pred == test_labels)
print('Test accuracy:', accuracy)

# Generate classification report
report = classification_report(test_labels, y_pred, target_names=["buildings","forest","glacier","mountain","sea","street"])
print(report)

# Save the final model
final_model.save_model('xgboost_model.json')

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [03:58:38] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Test accuracy: 0.17533333333333334
              precision    recall  f1-score   support

   buildings       0.15      0.14      0.15       437
      forest       0.17      0.15      0.16       474
     glacier       0.17      0.17      0.17       553
    mountain       0.20      0.25      0.22       525
         sea       0.16      0.15      0.16       510
      street       0.18      0.18      0.18       501

    accuracy                           0.18      3000
   macro avg       0.17      0.17      0.17      3000
weighted avg       0.17      0.18      0.17      3000



In [ ]:
# import os
# import numpy as np
# import xgboost as xgb
# from keras.models import model_from_json
# from keras import optimizers
# from keras.datasets import cifar10
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# import pickle

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# def load_cnn_model():
#     json_file = open('model_resnet.json', 'r')
#     loaded_model_json = json_file.read()
#     json_file.close()
#     model = model_from_json(loaded_model_json)
#     # model = model_from_json(loaded_model_json)
#     model.load_weights('model_resnet.weights.h5')
#     model.compile(
#         # optimizer=optimizers.RMSprop(learning_rate=2e-5),
#         loss='categorical_crossentropy',
#         metrics=['accuracy']
#     )

#     return model

# def get_feature_layer(model, data):
#     # Access the feature layer by name instead of index
#     feature_layer_model = keras.Model(
#         inputs=model.input,
#         outputs=model.get_layer(name='flatten').output # Change here
#     )
#     feature_layer_output = feature_layer_model.predict(data)

#     return feature_layer_output

# def xgb_model(X_train, y_train, X_test, y_test):

#     y_train = y_train.flatten()
#     y_test = y_test.flatten()

#     dtrain = xgb.DMatrix(
#         X_train,
#         label=y_train
#     )

#     dtest = xgb.DMatrix(
#         X_test,
#         label=y_test
#     )

#     params = {
#         # 'max_depth': 12,
#         # 'eta': 0.05,
#         'objective': 'multi:softprob',
#         'num_class': 10,
#         'early_stopping_rounds': 5,
#         'eval_metric': 'merror'
#     }

#     watchlist = [(dtrain, 'train'), (dtest, 'eval')]
#     # n_rounds = 100

#     model = xgb.train(
#         params,
#         dtrain,
#         # n_rounds,
#         evals=watchlist
#     )

#     return model

# def evaluate_model(model, X_test, y_test):
#     dtest = xgb.DMatrix(X_test)
#     y_pred_prob = model.predict(dtest)
#     y_pred = np.argmax(y_pred_prob, axis=1)

#     accuracy = accuracy_score(y_test, y_pred)
#     precision = precision_score(y_test, y_pred, average='macro')
#     recall = recall_score(y_test, y_pred, average='macro')
#     f1 = f1_score(y_test, y_pred, average='macro')

#     print(f'Accuracy: {accuracy}')
#     print(f'Precision: {precision}')
#     print(f'Recall: {recall}')
#     print(f'F1 Score: {f1}')

#     return accuracy, precision, recall, f1

# def main():
#     # Load CIFAR-10 dataset
#     (X_train, y_train), (X_test, y_test) = cifar10.load_data()

#     indices = np.random.permutation(X_train.shape[0])
#     train_size = 40000  # Ubah sesuai kebutuhan
#     X_train = X_train[indices][:train_size]
#     y_train = y_train[indices][:train_size]

#     # Normalize the data
#     X_train = X_train.astype('float32') / 255.0
#     X_test = X_test.astype('float32') / 255.0

#     y_train = y_train.flatten()
#     y_test = y_test.flatten()

#     # Load the CNN model and extract features
#     cnn_model = load_cnn_model()

#     X_train_cnn = get_feature_layer(cnn_model, X_train)
#     X_test_cnn = get_feature_layer(cnn_model, X_test)

#     # Train XGBoost model
#     model = xgb_model(X_train_cnn, y_train, X_test_cnn, y_test)

#     print("Build and save of XGBoost Model")
#     pickle.dump(model, open("cnn_xgboost_resnet_final.pickle.dat", "wb"))

#     # Evaluate the model
#     evaluate_model(model, X_test_cnn, y_test)

# if __name__ == '__main__':
#     main()


In [ ]:
import os
import numpy as np
import pickle
import xgboost as xgb
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import model_from_json
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from xgboost.callback import TrainingCallback
import keras

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

def load_cnn_model():
    json_file = open('model_resnet.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights('model_resnet.h5')

    model.compile(
        optimizer=optimizers.RMSprop(learning_rate=2e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

def get_feature_layer(model, data):
    total_layers = len(model.layers)
    fl_index = total_layers - 7  # Get the correct index for the feature layer

    feature_layer_model = keras.Model(
        inputs=model.input,
        outputs=model.get_layer(index=fl_index).output
    )
    feature_layer_output = feature_layer_model.predict(data)

    return feature_layer_output

def objective(space, X_train_cnn, y_train, X_valid_cnn, y_valid):
    params = {
        'learning_rate': space['learning_rate'],
        'max_depth': int(space['max_depth']),
        'min_child_weight': space['min_child_weight'],
        'subsample': space['subsample'],
        'colsample_bytree': space['colsample_bytree'],
        'base_score': space['base_score'],
        'colsample_bylevel': space['colsample_bylevel'],
        'colsample_bynode': space['colsample_bynode'],
        'num_parallel_tree': int(space['num_parallel_tree']),
        'max_delta_step': space['max_delta_step'],
        'gamma': space['gamma'],
        'objective': 'multi:softprob',
        'num_class': 10,
        'eval_metric': 'merror'
    }

    dtrain = xgb.DMatrix(X_train_cnn, label=y_train)
    dvalid = xgb.DMatrix(X_valid_cnn, label=y_valid)

    evals_result = {}
    model = xgb.train(params, dtrain, num_boost_round=1000, evals=[(dvalid, 'eval')],
                      early_stopping_rounds=10, evals_result=evals_result, verbose_eval=False)

    pred = model.predict(dvalid)
    pred_labels = np.argmax(pred, axis=1)
    accuracy = accuracy_score(y_valid, pred_labels)

    return {'loss': -accuracy, 'status': STATUS_OK}

def evaluate_model(model, X_test_cnn, y_test):
    dtest = xgb.DMatrix(X_test_cnn)
    pred = model.predict(dtest)
    pred_labels = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_test, pred_labels)
    precision = precision_score(y_test, pred_labels, average='weighted')
    recall = recall_score(y_test, pred_labels, average='weighted')
    f1 = f1_score(y_test, pred_labels, average='weighted')

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

class EarlyStopCallback(TrainingCallback):
    def __init__(self, threshold):
        self.threshold = threshold

    def after_iteration(self, model, epoch, evals_log):
        for eval_name, eval_metrics in evals_log.items():
            for metric_name, metric_values in eval_metrics.items():
                if eval_name == 'eval' and metric_name == 'merror' and 1 - metric_values[-1] > self.threshold:
                    print(f"Stopping early. Accuracy reached {1 - metric_values[-1]}")
                    return True
        return False

def main():
    # Load CIFAR-10 dataset
    (X_train, y_train), (X_test, y_test) = cifar10.load_data()

    # Preprocess data: normalize images
    X_train = X_train.astype('float32') / 255.0
    X_test = X_test.astype('float32') / 255.0

    # Data Augmentation
    datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True,
        zoom_range=0.2,
        shear_range=0.2
    )

    datagen.fit(X_train)

    # Convert labels to single-dimensional integers
    y_train = y_train.flatten()
    y_test = y_test.flatten()

    # Split training data into training and validation sets
    train_size = int(0.8 * len(X_train))
    X_train, X_valid = X_train[:train_size], X_train[train_size:]
    y_train, y_valid = y_train[:train_size], y_train[train_size:]

    # Load the CNN model and extract features
    cnn_model = load_cnn_model()

    X_train_cnn = get_feature_layer(cnn_model, X_train)
    X_valid_cnn = get_feature_layer(cnn_model, X_valid)
    X_test_cnn = get_feature_layer(cnn_model, X_test)

    # Define the search space
    space = {
        'learning_rate': hp.uniform('learning_rate', 0.001, 0.1),
        'max_depth': hp.quniform('max_depth', 1, 15, 1),
        'min_child_weight': hp.uniform('min_child_weight', 0, 5),
        'subsample': hp.uniform('subsample', 0.1, 1),
        'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 1),
        'base_score': hp.uniform('base_score', 0.1, 0.9),
        'colsample_bylevel': hp.uniform('colsample_bylevel', 0.1, 1),
        'colsample_bynode': hp.uniform('colsample_bynode', 0.1, 1),
        'max_delta_step': hp.uniform('max_delta_step', 0, 1),
        'num_parallel_tree': hp.quniform('num_parallel_tree', 1, 10, 1),
        'gamma': hp.uniform('gamma', 0, 0.1)
    }

    # Run the optimization
    trials = Trials()
    best = fmin(
        fn=lambda space: objective(space, X_train_cnn, y_train, X_valid_cnn, y_valid),
        space=space,
        algo=tpe.suggest,
        max_evals=100,
        trials=trials
    )

    print("Best hyperparameters found were:", best)

    # Train the final model with the best hyperparameters
    best_params = {
        'learning_rate': best['learning_rate'],
        'max_depth': int(best['max_depth']),
        'min_child_weight': best['min_child_weight'],
        'subsample': best['subsample'],
        'colsample_bytree': best['colsample_bytree'],
        'base_score': best['base_score'],
        'colsample_bylevel': best['colsample_bylevel'],
        'colsample_bynode': best['colsample_bynode'],
        'num_parallel_tree': int(best['num_parallel_tree']),
        'max_delta_step': best['max_delta_step'],
        'gamma': best['gamma'],
        'objective': 'multi:softprob',
        'num_class': 10,
        'eval_metric': 'merror',
        'booster': 'gbtree'
    }

    # Combine training and validation data
    X_combined_cnn = np.vstack((X_train_cnn, X_valid_cnn))
    y_combined = np.hstack((y_train, y_valid))

    dtrain_combined = xgb.DMatrix(X_combined_cnn, label=y_combined)

    # Define custom callback
    early_stop_callback = EarlyStopCallback(threshold=0.948)

    # Train the final model
    final_model = xgb.train(best_params, dtrain_combined, num_boost_round=1000, callbacks=[early_stop_callback])

    # Save the final model
    final_model.save_model('optimized_xgboost_model.dat')

    # Evaluate the final model
    evaluate_model(final_model, X_test_cnn, y_test)

if __name__ == '__main__':
    main()


In [ ]:
# import os
# import numpy as np
# import pickle
# import xgboost as xgb
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# from sklearn.model_selection import GridSearchCV
# from tensorflow.keras.datasets import cifar10
# from tensorflow.keras.models import model_from_json
# from tensorflow.keras import optimizers
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import keras

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# def load_cnn_model():
#     json_file = open('model_resnet.json', 'r')
#     loaded_model_json = json_file.read()
#     json_file.close()
#     model = model_from_json(loaded_model_json)
#     model.load_weights('model_resnet.h5')  # load weights into new model

#     model.compile(
#         optimizer=optimizers.RMSprop(learning_rate=2e-5),
#         loss='categorical_crossentropy',
#         metrics=['accuracy']
#     )

#     return model

# def get_feature_layer(model, data):
#     total_layers = len(model.layers)
#     fl_index = total_layers - 7  # Get the correct index for the feature layer

#     feature_layer_model = keras.Model(
#         inputs=model.input,
#         outputs=model.get_layer(index=fl_index).output
#     )
#     feature_layer_output = feature_layer_model.predict(data)

#     return feature_layer_output

# def evaluate_model(model, X_test_cnn, y_test):
#     dtest = xgb.DMatrix(X_test_cnn)
#     pred = model.predict(dtest)
#     pred_labels = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_test, pred_labels)
#     precision = precision_score(y_test, pred_labels, average='weighted')
#     recall = recall_score(y_test, pred_labels, average='weighted')
#     f1 = f1_score(y_test, pred_labels, average='weighted')

#     print(f"Accuracy: {accuracy}")
#     print(f"Precision: {precision}")
#     print(f"Recall: {recall}")
#     print(f"F1 Score: {f1}")

# def main():
#     # Load CIFAR-10 dataset
#     (X_train, y_train), (X_test, y_test) = cifar10.load_data()

#     # Preprocess data: normalize images
#     X_train = X_train.astype('float32') / 255.0
#     X_test = X_test.astype('float32') / 255.0

#     # Data Augmentation
#     datagen = ImageDataGenerator(
#         rotation_range=15,
#         width_shift_range=0.1,
#         height_shift_range=0.1,
#         horizontal_flip=True
#     )

#     datagen.fit(X_train)

#     # Convert labels to single-dimensional integers
#     y_train = y_train.flatten()
#     y_test = y_test.flatten()

#     # Split training data into training and validation sets
#     train_size = int(0.8 * len(X_train))
#     X_train, X_valid = X_train[:train_size], X_train[train_size:]
#     y_train, y_valid = y_train[:train_size], y_train[train_size:]

#     # Load the CNN model and extract features
#     cnn_model = load_cnn_model()

#     X_train_cnn = get_feature_layer(cnn_model, X_train)
#     X_valid_cnn = get_feature_layer(cnn_model, X_valid)
#     X_test_cnn = get_feature_layer(cnn_model, X_test)

#     # Combine training and validation data
#     X_combined_cnn = np.vstack((X_train_cnn, X_valid_cnn))
#     y_combined = np.hstack((y_train, y_valid))

#     # Define the parameter grid
#     param_grid = {
#         'learning_rate': [0.01, 0.1, 0.2],
#         'max_depth': [6, 10, 15],
#         'min_child_weight': [1, 5, 10],
#         'subsample': [0.8, 0.9, 1.0],
#         'colsample_bytree': [0.8, 0.9, 1.0],
#         'n_estimators': [100, 300, 500]
#     }

#     # Create DMatrix for combined training data
#     dtrain_combined = xgb.DMatrix(X_combined_cnn, label=y_combined)

#     # Initialize XGBoost model
#     xgb_model = xgb.XGBClassifier(objective='multi:softprob', num_class=10)

#     # Perform grid search
#     grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=1, n_jobs=-1)
#     grid_search.fit(X_combined_cnn, y_combined)

#     # Get the best parameters and best model
#     best_params = grid_search.best_params_
#     print("Best hyperparameters found were:", best_params)

#     # Train the final model with the best hyperparameters
#     final_model = xgb.XGBClassifier(**best_params, objective='multi:softprob', num_class=10)
#     final_model.fit(X_combined_cnn, y_combined)

#     # Save the final model
#     pickle.dump(final_model, open('optimized_xgboost_model_gridsearch.pkl', 'wb'))

#     # Evaluate the final model
#     evaluate_model(final_model.get_booster(), X_test_cnn, y_test)

# if __name__ == '__main__':
#     main()


In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical

# 1. Load CIFAR-10 dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# 2. Preprocess data: normalize images and flatten labels
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# Flatten labels
y_train = y_train.flatten()
y_test = y_test.flatten()

# 3. Define a function to extract features using a simple CNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

def create_feature_extractor():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Train a simple CNN to extract features
feature_extractor = create_feature_extractor()
feature_extractor.fit(X_train, to_categorical(y_train), epochs=10, batch_size=64, validation_split=0.2, verbose=2)

# Remove the last layer to get the feature extractor
feature_extractor = Sequential(feature_extractor.layers[:-1])

# 4. Extract features from the training and test data
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)

# 5. Train XGBoost model on extracted features
dtrain = xgb.DMatrix(X_train_features, label=y_train)
dtest = xgb.DMatrix(X_test_features, label=y_test)

params = {
    'objective': 'multi:softprob',
    'num_class': 10,
    'eval_metric': 'mlogloss',
    'max_depth': 6,
    'learning_rate': 0.1,
    'n_estimators': 100
}

bst = xgb.train(params, dtrain, num_boost_round=100)

# 6. Predict and evaluate the model
y_pred = bst.predict(dtest)
y_pred_labels = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, y_pred_labels)
precision = precision_score(y_test, y_pred_labels, average='weighted')
recall = recall_score(y_test, y_pred_labels, average='weighted')
f1 = f1_score(y_test, y_pred_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")


# DIPAKAI

In [14]:
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import optuna
import xgboost as xgb

# Define paths to dataset
train_dir = '/content/drive/MyDrive/intel/seg_train/seg_train'
test_dir = '/content/drive/MyDrive/intel/seg_test/seg_test'

# Data augmentation
datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_generator = datagen.flow_from_directory(train_dir, target_size=(150, 150), batch_size=256, class_mode='categorical')
test_generator = datagen.flow_from_directory(test_dir, target_size=(150, 150), batch_size=256, class_mode='categorical')

# Define class names
class_names = list(train_generator.class_indices.keys())
num_classes = len(class_names)


Found 14044 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


In [15]:
# Build the CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the CNN model
model.fit(train_generator, epochs=5, verbose=1, validation_data=test_generator)

# Save the CNN model
model.save('cnn_intel_image.h5')

# Evaluate CNN model
test_images, test_labels = next(test_generator)
cnn_preds = model.predict(test_images)
cnn_preds_classes = np.argmax(cnn_preds, axis=1)
true_classes = np.argmax(test_labels, axis=1)
cnn_accuracy = accuracy_score(true_classes, cnn_preds_classes)
cnn_precision = precision_score(true_classes, cnn_preds_classes, average='weighted')
cnn_recall = recall_score(true_classes, cnn_preds_classes, average='weighted')
cnn_f1 = f1_score(true_classes, cnn_preds_classes, average='weighted')
print(f'CNN Accuracy: {cnn_accuracy}')
print(f'CNN Precision: {cnn_precision}')
print(f'CNN Recall: {cnn_recall}')
print(f'CNN F1-Score: {cnn_f1}')


Epoch 1/5
55/55 [==============================] - 84s 1s/step - loss: 1.2607 - accuracy: 0.5140 - val_loss: 0.9450 - val_accuracy: 0.6443
Epoch 2/5
55/55 [==============================] - 74s 1s/step - loss: 0.8266 - accuracy: 0.6888 - val_loss: 0.7577 - val_accuracy: 0.7213
Epoch 3/5
55/55 [==============================] - 78s 1s/step - loss: 0.6852 - accuracy: 0.7503 - val_loss: 0.6304 - val_accuracy: 0.7760
Epoch 4/5
55/55 [==============================] - 72s 1s/step - loss: 0.5779 - accuracy: 0.7925 - val_loss: 0.5460 - val_accuracy: 0.8017
Epoch 5/5
55/55 [==============================] - 74s 1s/step - loss: 0.5011 - accuracy: 0.8238 - val_loss: 0.6055 - val_accuracy: 0.7793


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


8/8 [==============================] - 1s 11ms/step
CNN Accuracy: 0.7265625
CNN Precision: 0.765094429560388
CNN Recall: 0.7265625
CNN F1-Score: 0.7287589059443118


In [16]:
# Load the CNN model
model = load_model('cnn_intel_image.h5')

# Create a feature extractor model
feature_extractor = Model(inputs=model.inputs, outputs=model.layers[-3].output)

# Function to extract features
def extract_features(generator, model):
    features = []
    labels = []
    for inputs_batch, labels_batch in generator:
        features_batch = model.predict(inputs_batch)
        features.append(features_batch)
        labels.append(labels_batch)
        if len(features) * generator.batch_size >= generator.samples:
            break
    return np.concatenate(features), np.concatenate(labels)

# Extract features for training and test sets
train_features, train_labels = extract_features(train_generator, feature_extractor)
test_features, test_labels = extract_features(test_generator, feature_extractor)

# Convert labels to one-dimensional array
train_labels_flat = np.argmax(train_labels, axis=1)
test_labels_flat = np.argmax(test_labels, axis=1)

# Define XGBoost model parameters (without TPE)
xgb_params = {
    'objective': 'multi:softmax',
    'num_class': num_classes,
    'max_depth': 8,
    'learning_rate': 0.1,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
}

# Train the XGBoost model without TPE
xgb_model = xgb.XGBClassifier(**xgb_params)
xgb_model.fit(train_features, train_labels_flat)

# Evaluate XGBoost model without TPE
xgb_preds = xgb_model.predict(test_features)
xgb_accuracy = accuracy_score(test_labels_flat, xgb_preds)
xgb_precision = precision_score(test_labels_flat, xgb_preds, average='weighted')
xgb_recall = recall_score(test_labels_flat, xgb_preds, average='weighted')
xgb_f1 = f1_score(test_labels_flat, xgb_preds, average='weighted')
print(f'XGBoost Accuracy: {xgb_accuracy}')
print(f'XGBoost Precision: {xgb_precision}')
print(f'XGBoost Recall: {xgb_recall}')
print(f'XGBoost F1-Score: {xgb_f1}')


8/8 [==============================] - 0s 6ms/step
XGBoost Accuracy: 0.83
XGBoost Precision: 0.8298612077136445
XGBoost Recall: 0.83
XGBoost F1-Score: 0.8298795477915067


In [19]:
# Load the CNN model
model = load_model('cnn_intel_image.h5')

# Create a feature extractor model
feature_extractor = Model(inputs=model.inputs, outputs=model.layers[-3].output)

# Extract features for training and test sets
train_features, train_labels = extract_features(train_generator, feature_extractor)
test_features, test_labels = extract_features(test_generator, feature_extractor)

# Convert labels to one-dimensional array
train_labels_flat = np.argmax(train_labels, axis=1)
test_labels_flat = np.argmax(test_labels, axis=1)

# Define objective function for Optuna (with TPE)
def objective(trial):
    param = {
        'objective': 'multi:softmax',
        'num_class': num_classes,
        'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 1, 15),
        'min_child_weight': trial.suggest_uniform('min_child_weight', 0, 5),
        'subsample': trial.suggest_uniform('subsample', 0.1, 1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.1, 1),
        'base_score': trial.suggest_uniform('base_score', 0.1, 0.9),
        'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.1, 1),
        'colsample_bynode': trial.suggest_uniform('colsample_bynode', 0.1, 1),
        'max_delta_step': trial.suggest_uniform('max_delta_step', 0, 1),
        'num_parallel_tree': trial.suggest_int('num_parallel_tree', 1, 10),
        'gamma': trial.suggest_uniform('gamma', 0, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
    }

    model = xgb.XGBClassifier(**param)
    model.fit(train_features, train_labels_flat)

    preds = model.predict(test_features)
    accuracy = accuracy_score(test_labels_flat, preds)
    return accuracy

# Create study and optimize
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=5)

# Train the final model with the best parameters (with TPE)
best_params = study.best_params
final_model = xgb.XGBClassifier(**best_params)
final_model.fit(train_features, train_labels_flat)

# Evaluate the model with TPE
tpe_preds = final_model.predict(test_features)
tpe_accuracy = accuracy_score(test_labels_flat, tpe_preds)
tpe_precision = precision_score(test_labels_flat, tpe_preds, average='weighted')
tpe_recall = recall_score(test_labels_flat, tpe_preds, average='weighted')
tpe_f1 = f1_score(test_labels_flat, tpe_preds, average='weighted')
print(f'XGBoost with TPE Accuracy: {tpe_accuracy}')
print(f'XGBoost with TPE Precision: {tpe_precision}')
print(f'XGBoost with TPE Recall: {tpe_recall}')
print(f'XGBoost with TPE F1-Score: {tpe_f1}')


8/8 [==============================] - 0s 7ms/step


[I 2024-08-08 08:17:58,617] A new study created in memory with name: no-name-a8aaf9ba-cf6b-4989-9aa6-189f793c216b
<ipython-input-19-a5c6f79c9aa4>:20: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),
<ipython-input-19-a5c6f79c9aa4>:22: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_child_weight': trial.suggest_uniform('min_child_weight', 0, 5),
<ipython-input-19-a5c6f79c9aa4>:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.1, 1),
<ipython-input-1

XGBoost with TPE Accuracy: 0.8236666666666667
XGBoost with TPE Precision: 0.8239354174586377
XGBoost with TPE Recall: 0.8236666666666667
XGBoost with TPE F1-Score: 0.8236713372549874


In [20]:
from keras.models import Model, load_model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import optuna
import xgboost as xgb
import numpy as np

# Fungsi untuk mengekstraksi fitur
def extract_features(generator, model):
    features = []
    labels = []
    for i in range(len(generator)):
        x, y = generator[i]
        feature = model.predict(x)
        features.append(feature)
        labels.append(y)
    return np.concatenate(features), np.concatenate(labels)

# Load the CNN model
model = load_model('cnn_intel_image.h5')

# Create a feature extractor model
feature_extractor = Model(inputs=model.inputs, outputs=model.layers[-3].output)

# Extract features for training and test sets
train_features, train_labels = extract_features(train_generator, feature_extractor)
test_features, test_labels = extract_features(test_generator, feature_extractor)

# Convert labels to one-dimensional array
train_labels_flat = np.argmax(train_labels, axis=1)
test_labels_flat = np.argmax(test_labels, axis=1)

# Define objective function for Optuna (with TPE)
def objective(trial):
    param = {
        'objective': 'multi:softmax',
        'num_class': len(np.unique(train_labels_flat)),
        'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_weight': trial.suggest_uniform('min_child_weight', 1, 10),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1),
        'gamma': trial.suggest_uniform('gamma', 0, 0.5),
        'alpha': trial.suggest_uniform('alpha', 0, 0.5),
        'lambda': trial.suggest_uniform('lambda', 0, 0.5),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
    }

    model = xgb.XGBClassifier(**param, use_label_encoder=False, eval_metric='mlogloss')
    model.fit(train_features, train_labels_flat)

    preds = model.predict(test_features)
    accuracy = accuracy_score(test_labels_flat, preds)
    return accuracy

# Create study and optimize
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler())
study.optimize(objective, n_trials=5)  # Meningkatkan jumlah trials

# Train the final model with the best parameters (with TPE)
best_params = study.best_params
final_model = xgb.XGBClassifier(**best_params, use_label_encoder=False, eval_metric='mlogloss')
final_model.fit(train_features, train_labels_flat)

# Evaluate the model with TPE
tpe_preds = final_model.predict(test_features)
tpe_accuracy = accuracy_score(test_labels_flat, tpe_preds)
tpe_precision = precision_score(test_labels_flat, tpe_preds, average='weighted')
tpe_recall = recall_score(test_labels_flat, tpe_preds, average='weighted')
tpe_f1 = f1_score(test_labels_flat, tpe_preds, average='weighted')
print(f'XGBoost with TPE Accuracy: {tpe_accuracy}')
print(f'XGBoost with TPE Precision: {tpe_precision}')
print(f'XGBoost with TPE Recall: {tpe_recall}')
print(f'XGBoost with TPE F1-Score: {tpe_f1}')


6/6 [==============================] - 0s 8ms/step


[I 2024-08-08 09:22:23,961] A new study created in memory with name: no-name-361212fd-8582-4d8e-a5ad-903dca0f4401
<ipython-input-20-1ad406fad8c1>:37: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'learning_rate': trial.suggest_uniform('learning_rate', 0.001, 0.1),
<ipython-input-20-1ad406fad8c1>:39: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'min_child_weight': trial.suggest_uniform('min_child_weight', 1, 10),
<ipython-input-20-1ad406fad8c1>:40: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample': trial.suggest_uniform('subsample', 0.5, 1),
<ipython-input-

XGBoost with TPE Accuracy: 0.8333333333333334
XGBoost with TPE Precision: 0.8333387299610068
XGBoost with TPE Recall: 0.8333333333333334
XGBoost with TPE F1-Score: 0.8332496637738485


In [23]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from xgboost import XGBClassifier

# Path ke dataset
dataset_path_train = '/content/drive/MyDrive/intel/seg_train/'
dataset_path_test = '/content/drive/MyDrive/intel/seg_test/'

# Kelas yang ada di dataset
classes = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']

# Fungsi untuk membaca dan preprocess gambar
def load_images_from_folder(folder):
    images = []
    labels = []
    for class_idx, class_name in enumerate(classes):
        class_folder = os.path.join(folder, class_name)
        for filename in os.listdir(class_folder):
            img_path = os.path.join(class_folder, filename)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.resize(img, (64, 64))  # Resize gambar ke ukuran yang sama
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Ubah ke grayscale
                images.append(img)
                labels.append(class_idx)
    return np.array(images), np.array(labels)

# Load data train dan test
train_images, train_labels = load_images_from_folder(os.path.join(dataset_path_train, 'seg_train'))
test_images, test_labels = load_images_from_folder(os.path.join(dataset_path_test, 'seg_test'))

# Flatten gambar untuk ekstraksi fitur
train_images_flatten = train_images.reshape(train_images.shape[0], -1)
test_images_flatten = test_images.reshape(test_images.shape[0], -1)

# Split data train menjadi train dan validation
X_train, X_val, y_train, y_val = train_test_split(train_images_flatten, train_labels, test_size=0.2, random_state=42)

# Buat dan latih model XGBoost
model = XGBClassifier()
model.fit(X_train, y_train)

# Prediksi pada data validation
y_val_pred = model.predict(X_val)
val_accuracy = accuracy_score(y_val, y_val_pred)
val_precision = precision_score(y_val, y_val_pred, average='weighted')
val_recall = recall_score(y_val, y_val_pred, average='weighted')
val_f1 = f1_score(y_val, y_val_pred, average='weighted')

print(f'Validation Accuracy: {val_accuracy * 100:.2f}%')
print(f'Validation Precision: {val_precision * 100:.2f}%')
print(f'Validation Recall: {val_recall * 100:.2f}%')
print(f'Validation F1-Score: {val_f1 * 100:.2f}%')

# Prediksi pada data test
y_test_pred = model.predict(test_images_flatten)
test_accuracy = accuracy_score(test_labels, y_test_pred)
test_precision = precision_score(test_labels, y_test_pred, average='weighted')
test_recall = recall_score(test_labels, y_test_pred, average='weighted')
test_f1 = f1_score(test_labels, y_test_pred, average='weighted')

print(f'Test Accuracy: {test_accuracy * 100:.2f}%')
print(f'Test Precision: {test_precision * 100:.2f}%')
print(f'Test Recall: {test_recall * 100:.2f}%')
print(f'Test F1-Score: {test_f1 * 100:.2f}%')


Validation Accuracy: 58.81%
Validation Precision: 58.66%
Validation Recall: 58.81%
Validation F1-Score: 58.33%
Test Accuracy: 57.93%
Test Precision: 57.85%
Test Recall: 57.93%
Test F1-Score: 57.56%
